# 11. 항공 사진 내 선인장 식별
## 11.4 성능 개선
* 다양한 이미지 변환 수행
* 더 깊은 CNN 모델 구성
* 뛰어난 옵티마이저 사용
* 훈련시 에폭수 늘림

### 11.4.1 데이터 준비
* data augmentation
    * torchvision의 transforms 모듈
        * Compose() : 여러 변환기를 묶어줌
        * ToTensor() : PIL 이미지나 ndarray를 텐서로 변환
        * Pad() : 이미지 주변에 패딩 추가
        * RandomHorizontalFlip() : 이미지를 무작위로 좌우 대칭 변환
        * RandomVerticalFlip() : 이미지를 무작위로 상하 대칭 변환
        * RandomRotation() : 이미지를 무작위로 회전
        * Normalize() : 텐서 형태의 이미지 데이터를 정규화

In [5]:
from torchvision import transforms

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32, padding_mode='symmetric'), # 32로 지정했으므로 원본포함 총 9장의 이미지로 변환, symmetric - 상하좌우대칭으로 생성
    transforms.RandomHorizontalFlip(), # 좌우 대칭 변환, 50% 확률
    transforms.RandomVerticalFlip(), # 상하 대칭 변환, 50% 확률
    transforms.RandomRotation(10), # -10~10도 이미지 회전, 50% 확률
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # 이미지넷으로부터 얻은 수치
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32, padding_mode='symmetric'), # 32로 지정했으므로 원본포함 총 9장의 이미지로 변환, symmetric - 상하좌우대칭으로 생성
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # 이미지넷으로부터 얻은 수치
])

In [6]:
from zipfile import ZipFile
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

data_path = '../../data/11_cactus/'
labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

with ZipFile(data_path+'train.zip') as zipper:
    zipper.extractall(data_path)
with ZipFile(data_path+'test.zip') as zipper:
    zipper.extractall(data_path)

# 훈련/검증 데이터 분리
train, valid = train_test_split(
    labels,
    test_size=0.1,
    stratify=labels['has_cactus'],
    random_state=50
)

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform if transform else lambda x: x

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]
        image = self.transform(image)
        return image, label

dataset_train = ImageDataset(df=train, img_dir=data_path+'train/', transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir=data_path+'train/', transform=transform_test)
loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

* 에폭마다 서로 다른 이미지로 훈련하는 효과를 가짐

### 11.4.2 모델 생성

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.fc1 = nn.Linear(in_features=512*1*1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512*1*1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [8]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyModel().to(device)

### 11.4.3 모델 훈련

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006)

In [10]:
epochs = 70
for epoch in range(epochs):
    epoch_loss = 0
    for images, labels in loader_train:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')

에폭 [1/70] - 손실값: 0.1336
에폭 [2/70] - 손실값: 0.0721
에폭 [3/70] - 손실값: 0.0566
에폭 [4/70] - 손실값: 0.0478
에폭 [5/70] - 손실값: 0.0417
에폭 [6/70] - 손실값: 0.0347
에폭 [7/70] - 손실값: 0.0316
에폭 [8/70] - 손실값: 0.0307
에폭 [9/70] - 손실값: 0.0282
에폭 [10/70] - 손실값: 0.0269
에폭 [11/70] - 손실값: 0.0227
에폭 [12/70] - 손실값: 0.0250
에폭 [13/70] - 손실값: 0.0202
에폭 [14/70] - 손실값: 0.0205
에폭 [15/70] - 손실값: 0.0185
에폭 [16/70] - 손실값: 0.0184
에폭 [17/70] - 손실값: 0.0178
에폭 [18/70] - 손실값: 0.0173
에폭 [19/70] - 손실값: 0.0161
에폭 [20/70] - 손실값: 0.0139
에폭 [21/70] - 손실값: 0.0152


KeyboardInterrupt: 

### 11.4.4 성능 검증

In [ ]:
from sklearn.metrics import roc_auc_score
true_list = []
preds_list = []
model.eval()
with torch.no_grad():
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu()
        preds_list.extends(preds)
        true_list.extend(true)

print(f'Valid ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

### 11.4.5 예측 및 결과 제출

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir=data_path+'test/', transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)
model.eval()
preds = []
with torch.no_grad():
    for images, _ in loader_test:
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        preds.extend(preds_part)

In [ ]:
submission['has_cactus'] = preds
submission.to_csv(data_path+'submission.csv', index=False)

In [ ]:
import shutil

shutil.rmtree(data_path+'train')
shutil.rmtree(data_path+'test')